In [1]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 3.6 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.7.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
fashion_mnist

<module 'keras.api._v2.keras.datasets.fashion_mnist' from '/usr/local/lib/python3.7/dist-packages/keras/api/_v2/keras/datasets/fashion_mnist/__init__.py'>

In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [7]:
train_images

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [8]:
train_labels

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [9]:
train_images.shape

(60000, 28, 28)

In [10]:
train_images = train_images/255.0
test_images = test_images/255.0

In [11]:
train_images.shape

(60000, 28, 28)

In [12]:
train_images[0].shape

(28, 28)

In [13]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [14]:
train_images.shape

(60000, 28, 28, 1)

In [15]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [16]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [17]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [18]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 17m 28s]
val_accuracy: 0.8736666440963745

Best val_accuracy So Far: 0.9081666469573975
Total elapsed time: 01h 21m 54s
INFO:tensorflow:Oracle triggered exit


In [19]:
model=tuner_search.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        27696     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 64)                1769536   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,798,522
Trainable params: 1,798,522
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 204s 121ms/step - loss: 0.1310 - accuracy: 0.9514 - val_loss: 0.2770 - val_accuracy: 0.9058
Epoch 5/10
1688/1688 [==============================] - 205s 121ms/step - loss: 0.0914 - accuracy: 0.9664 - val_loss: 0.2943 - val_accuracy: 0.9073
Epoch 6/10
1688/1688 [==============================] - 205s 121ms/step - loss: 0.0634 - accuracy: 0.9763 - val_loss: 0.3826 - val_accuracy: 0.9138
Epoch 7/10
1688/1688 [==============================] - 205s 121ms/step - loss: 0.0464 - accuracy: 0.9829 - val_loss: 0.3897 - val_accuracy: 0.9127
Epoch 8/10
1688/1688 [==============================] - 202s 120ms/step - loss: 0.0332 - accuracy: 0.9880 - val_loss: 0.4384 - val_accuracy: 0.9153
Epoch 9/10
1688/1688 [==============================] - 202s 120ms/step - loss: 0.0270 - accuracy: 0.9901 - val_loss: 0.4517 - val_accuracy: 0.9112
Epoch 10/10
1688/1688 [==============================] - 203s 120ms/step - loss: 0.0238 - accuracy: 0.9917 - val